In [2]:
#COpilot langserve client
from langchain.agents import AgentExecutor, tool
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.tools import StructuredTool


In [3]:
from langserve import RemoteRunnable
host = "http://localhost:8000/"

copilotv=RemoteRunnable(host+"cotalk/")
copilotg=RemoteRunnable(host+"kg/")
copilotrag = RemoteRunnable(host+"rag/")
copilotcombines = RemoteRunnable(host+"crag/")

In [53]:
import requests
from cachetools import cached, TTLCache

# Create a cache with a maximum of 100 entries and a time-to-live (TTL) of 180 seconds
cache = TTLCache(maxsize=100, ttl=180)

@cached(cache)
def rag_server_response(query,host="http://localhost:8000/rag",key="query"):
    import requests 
    import json
    resp = requests.get(host+"?"+key+"="+query)
    if resp.status_code==200:
        return json.loads(resp.text)

@cached(cache)
def rag_server_response_stream(query, host="http://localhost:8000/rag", key="query"):
    try:
        resp = requests.get(host, params={key: query}, stream=True)
        resp.raise_for_status()  # Raise an exception if the response status code is not 200
        for chunk in resp.iter_content(chunk_size=1024):  # Adjust chunk size as needed
            yield chunk
    except requests.RequestException as e:
        print(f"Error fetching data: {e}")


#Simple function to
def vector_answers(query:str,qkey="question",chain=copilotv)->list:
    """Gets answers using vector RAG.
    Args:
        query: A string containing the question to ask.
        qkey: A string indicating the key for the question in the input dictionary. Default is "question".
        chain: A RemoteRunnable object that represents the vector RAG model.
    Returns:
        A list of answers from the vector RAG model.
    """
    
    return chain({"question":query}.get("answer"))
    

def RAG_asnwers_tool():
    structured_tool = StructuredTool.from_function(
        name="Get LLM response using vector or graph RAG tool",
        func=copilot_response_server,
        description="Copilot use this tool if you want to get a RAG response using knowledge graph or vector embeddings from business documents by API provisioning LangServe")

    return structured_tool

# from langchain.agents import tool


# @tool
# def get_word_length(word: str) -> int:
#     """Returns the length of a word."""
#     return len(word)


# @tool
# def graph_response(query):
#     pass

async def stream_obj_response(copilotobj,query,qkey="question"):
    "Streaming response using async stream"
    final_response = []
    async for msg in copilotobj.astream({qkey:query}):
        print(msg.get("answer"), end="", flush=True)
        final_response.append(msg.get("answer"), end="", flush=True)
    return final_response
            
@cached(cache)
def copilot_response_server(query:str, qkey="question",fullrag=False,copilotobj=None,stream=False,**kwargs):
    "Gets answers using vector RAG from vectors, graphs or RAG response class"
    if fullrag:
        host = kwargs.get("host","http://localhost:8000/rag")
        if stream:
            r = rag_server_response_stream(query,host=host)
        else:
            r = rag_server_response(query,host=host)
    else:
        #qkey for vector question and for graph 
        if stream: #streaming response
            r = stream_obj_response(copilotobj,query,qkey)
        else:
            r=copilotobj.invoke({qkey:query})
    return r


## Testing Vector Retriever in Langserve

In [5]:
# import asyncio
# asyncio.run(stream_obj_response(copilotobj=copilotv,query="how many tickets are related to benchmill"))

In [6]:
copilot_response_server("how many tickets are related to benchmill",copilotobj=copilotv)

{'answer': 'Based on the provided incident tickets, there is one ticket (INC0007654) that is related to the Benchmill.'}

In [7]:
copilot_response_server("what are the others about then",copilotobj=copilotv)

{'answer': 'The other pieces of information provided are incident tickets that describe specific issues or incidents that occurred in different production lines. These incidents include variations in the quality and accuracy of produced machined parts, unexpected shutdown of a machine impacting the production schedule, excessive vibrations or unusual noise leading to poor product quality, and a sudden and unplanned production stoppage. The parameter values associated with each incident provide additional details about the operating conditions and maintenance history of the machines involved.'}

In [8]:
copilot_response_server("can you perform an individual root cause analysis and give me a plan of action for each of them",copilotobj=copilotv)

{'answer': 'For Incident Ticket ID: INC0007654, the likely root cause of the unexpected shutdown of Benchmill on Production Line 2 is inadequate cooling leading to motor overheating. This is indicated by the breach in standard values for the airflow rate (70 CFM instead of >50 CFM), ambient temperature (35 degrees Celsius instead of 20-25 degrees Celsius), and motor surface temperature (80 degrees Celsius instead of <60 degrees Celsius).\n\nPlan of Action:\n1) Check and clean the cooling vents to ensure proper airflow.\n2) Implement measures to reduce the ambient temperature around the motor, such as improving ventilation or installing cooling systems.\n3) Investigate and address any issues causing the motor surface temperature to exceed the standard value, such as faulty cooling mechanisms or excessive workload on the motor.\n\nFor Incident Ticket ID: INC0003557, the likely root cause of the sudden production stoppage on Assembly Line 3 is insufficient power supply causing fluctuation

#### Vector Retriever Stream response

In [38]:
#as stream response
async for msg in copilotv.astream({"question":"can you perform an individual root cause analysis for ticket 1"}):
    print(msg.get("answer"), end="", flush=True)

Based on the provided information, the individual root cause analysis for Incident Ticket ID INC0007654 is as follows:

Root Cause: Unexpected shutdown of Benchmill
Possible Causes:
1. Motor surface temperature: The motor surface temperature of 80 degrees Celsius may have exceeded the safe operating limit, leading to the shutdown.
2. Load on motor: The motor was operating at 70% load, which may have caused overheating and triggered the shutdown.
3. Voltage stability: A 5% voltage stability may have caused fluctuations in the power supply, leading to the shutdown.
4. Power wiring resistance: A power wiring resistance of 0.01 ohms may have caused excessive heat generation and triggered the shutdown.
5. Dust accumulation: A dust accumulation of 0.5 mm thickness may have affected the motor's performance and caused the shutdown.

Recommended Actions:
1. Monitor and control motor surface temperature to ensure it stays within the safe operating range.
2. Evaluate the load on the motor and con

## Testing GraphQA Retriever in Langserver

In [15]:
copilot_response_server("what did the study concluded",copilotobj=copilotg)

{'result': 'The study concluded that only about 13% of methane emissions are covered by methane mitigation policies. Additionally, the effectiveness of these policies is far from clear.'}

In [14]:
#RUN examples
copilot_response_server("tell me everything you know about",copilotobj=copilotg)

{'result': "I know several pieces of information about different topics. Here are some of them:\n\n1. There are theories of planetary formation that have been challenged.\n2. There is a lowest-mass star hosting something, although it is not specified what it is hosting.\n3. Five employees have published a tropical cyclone report.\n4. In August, there was a new record high temperature reached.\n5. The world's oceans have also reached a new record high temperature.\n6. Hurricane Ian was upgraded from Category 4 to Category 5.\n7. Hurricane Ian caused $112.9 billion worth of damage.\n8. Hurricane Ian struck Florida.\n9. The oxygen partial pressure is above 18%.\n10. Emissions of chlorofluorocarbons have risen.\n\nPlease let me know if you would like more information on any specific topic."}

In [16]:
copilot_response_server("tell me about the earths heaviest animal",copilotobj=copilotg)

{'result': "The Earth's heaviest animal is the blue whale. It can weigh up to 200 tons, which is equivalent to about 400,000 pounds. These magnificent creatures can grow to be over 100 feet long and have a heart that weighs as much as a car. Despite their massive size, blue whales primarily feed on tiny shrimp-like animals called krill."}

## RAG responses

In [91]:
#RESPONSES from messages without memory

In [18]:
import requests
import json

In [58]:
#RESPONSE
copilot_response_server("Perform FMEA on first ticket dats",fullrag=True)

{'result': 'To perform FMEA on the data from the first incident ticket, we need to analyze the potential failure modes, their effects, and the likelihood of their occurrence. \n\n1. Failure Mode: Unexpected shutdown of Benchmill\n   - Effect: Disruption of production schedule\n   - Occurrence: Unknown\n\n2. Failure Mode: Airflow Rate below normal\n   - Effect: Decreased cooling efficiency\n   - Occurrence: Unlikely\n\n3. Failure Mode: High Ambient Temperature\n   - Effect: Increased motor temperature\n   - Occurrence: Unlikely\n\n4. Failure Mode: High Motor Surface Temperature\n   - Effect: Potential overheating and damage to the motor\n   - Occurrence: Unlikely\n\n5. Failure Mode: High Load on Motor\n   - Effect: Increased stress on the motor\n   - Occurrence: Unlikely\n\n6. Failure Mode: Low User Workload\n   - Effect: Inadequate utilization of the motor\n   - Occurrence: Unlikely\n\n7. Failure Mode: Voltage Stability Issues\n   - Effect: Fluctuations in motor performance\n   - Occur

In [20]:
copilot_response_server("Perform RCA on first ticket data",fullrag=True)

<Response [200]>


{'result': 'Root Cause Analysis (RCA) for Incident Ticket ID: INC0007654:\n\nIssue Description: Production Line 2 experienced a sudden disruption during the morning shift due to unexpected shutdown of Benchmill impacting the production schedule.\n\nParameter Values:\n- Airflow Rate: 70 CFM\n- Ambient Temperature: 35 degree celsius\n- Motor surface temperature: 80 degree celsius\n- Motor rated power: 1.5 HP\n- Load on motor: 70%\n- User workload: 60%\n- Voltage stability: 5%\n- Power wiring resistance: 0.01 ohms\n- Voltage fluctuation: 1%\n- Maintenance logs show monthly frequency\n- Dust accumulation: 0.5 mm thickness\n- Lubrication frequency: 50 hours\n\nBased on the provided data, the potential root causes for the unexpected shutdown of Benchmill on Production Line 2 can be identified as follows:\n\n1. Overheating of the motor: The motor surface temperature of 80 degree celsius indicates that the motor might have been operating at a higher temperature than normal. This could be due t

In [33]:
copilot_response_server("What did all the studies concluded",fullrag=True,method="graph")
print("As we can see there is no information from knowledge graph base")

As we can see there is no information from knowledge graph base


This RESPONSE has no memory

## Agent executor tools LLMS (langchain vector and graph)

In [47]:
print("VECTOR chain RAG from agent executor")
copilot_response_server("perform an extensive and professional RCA for ticket 1 then give recommendations",host="http://localhost:8000/crag",fullrag=True)

VECTOR chain RAG from agent executor


'Based on the provided incident ticket (INC0007654) and the reference values, the following RCA can be performed:\n\n1) Cooling mechanisms: The airflow rate through the cooling vents is 70 CFM, which exceeds the standard value of 50 CFM. This indicates that the cooling is adequate.\n\n2) Ambient temperature: The ambient temperature around the motor is 35 degrees Celsius, which exceeds the standard value of 20-25 degrees Celsius. This suggests that the motor is operating in a higher temperature environment.\n\n3) Motor surface temperature: The motor surface temperature during operation is 80 degrees Celsius, which exceeds the standard value of 60 degrees Celsius. This indicates that the motor is overheating.\n\nBased on the above analysis, the likely root cause of the unexpected shutdown of the Benchmill is inadequate cooling leading to motor overheating. \n\nRecommendations:\n1) Improve the cooling mechanisms by ensuring proper airflow through the cooling vents. This can be achieved by

In [51]:
print("GRAPH chain RAG from agent executor")
copilot_response_server("tell me everything you know about",host="http://localhost:8000/crag",fullrag=True)

GRAPH chain RAG from agent executor


"Here is some information that I know about different topics:\n\n1. There are theories of planetary formation that have been challenged.\n2. There is a lowest-mass star hosting something, although it is not specified what it is hosting.\n3. Five employees have published a tropical cyclone report.\n4. In August, there was a new record high temperature reached.\n5. The world's oceans have also reached a new record high temperature.\n6. Hurricane Ian was upgraded from Category 4 to Category 5.\n7. Hurricane Ian caused $112.9 billion worth of damage.\n8. Hurricane Ian struck Florida.\n9. The oxygen partial pressure is above 18%.\n10. Emissions of chlorofluorocarbons have risen.\n\nIf you have any specific questions or need more information on any of these topics, please let me know."

In [52]:
copilot_response_server("what did all the studies concluded",host="http://localhost:8000/crag",fullrag=True)

'Based on the information from the Copilot_Knowledge function, all the studies concluded that only about 13% of methane emissions are covered by methane mitigation policies. The effectiveness of these policies is also uncertain.'

In [37]:
copilot_response_server("how can you help me with my manufacturing industry",host="http://localhost:8000/crag",fullrag=True)

"I can assist you with your manufacturing industry by providing guidance and support in various areas. This includes optimizing your production processes, improving efficiency, implementing quality control measures, and identifying opportunities for cost savings. Additionally, I can help you stay updated with industry trends and advancements, as well as provide insights on best practices. Let me know specifically what you need assistance with, and I'll be happy to help."

In [61]:
#response in streaming
for c in copilot_response_server("Perform FMEA on first ticket data",fullrag=True,stream=True):
    print(c.decode("utf-8"))    
# for chunk in rag_server_response_stream("list all the incident tickets and give the recommended actions"):
#     print(chunk.decode("utf-8"))


{"result":"To perform FMEA on the data from the first incident ticket, we need to analyze the potential failure modes, their effects, and the likelihood of their occurrence. \n\n1. Failure Mode: Unexpected shutdown of Benchmill\n   - Effect: Disruption of production schedule\n   - Occurrence: Unknown\n\n2. Failure Mode: Airflow Rate below normal\n   - Effect: Decreased cooling efficiency\n   - Occurrence: Unlikely\n\n3. Failure Mode: High Ambient Temperature\n   - Effect: Increased motor temperature\n   - Occurrence: Unlikely\n\n4. Failure Mode: High Motor Surface Temperature\n   - Effect: Potential overheating and damage to the motor\n   - Occurrence: Unlikely\n\n5. Failure Mode: Overloaded Motor\n   - Effect: Increased stress on the motor\n   - Occurrence: Unlikely\n\n6. Failure Mode: Low Voltage Stability\n   - Effect: Inconsistent motor performance\n   - Occurrence: Unlikely\n\n7. Failure Mode: High Power Wiring Resistance\n   - Effect: Increased power loss and heat generation\n   

### Now we need to test the agent executor in conversational way and also use llama index as a tool

In [26]:
# from langchain.agents import tool
# @tool
# def get_word_length(word: str) -> int:
#     """Returns the length of a word."""
#     return len(word)

# tools = [get_word_length]

# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# prompt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             "You are very powerful assistant, but don't know current events",
#         ),
#         ("user", "{input}"),
#         MessagesPlaceholder(variable_name="agent_scratchpad"),
#     ]
# )

In [27]:
# llm_with_tools = llmrca.bind_tools(tools)



In [28]:
# from langchain.agents.format_scratchpad.openai_tools import (
#     format_to_openai_tool_messages,
# )
# from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

# agent = (
#     {
#         "input": lambda x: x["input"],
#         "agent_scratchpad": lambda x: format_to_openai_tool_messages(
#             x["intermediate_steps"]
#         ),
#     }
#     | prompt
#     | llm_with_tools
#     | OpenAIToolsAgentOutputParser()
# )

# from langchain.agents import AgentExecutor

# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [93]:
# list(agent_executor.invoke({"input": "How many letters in the word eudca"}))



In [29]:
# llm_with_tools.invoke("How many letters in the word eudca")